# Sistema especialista manutenção de hardware - Placa mãe

## Instalação do Experta

In [119]:
# Instalação do experta caso n esteja instalado
%pip install experta

Note: you may need to restart the kernel to use updated packages.


## 1 - Definindo os fatos

In [120]:
from experta import * # type: ignore

class PlacaMae(Fact):
    pass

## 2 - Sistema Especialista

As regras foram construídas com base na documentação de fabricantes de BIOS/placas-mãe (AMI e Gigabyte).
Durante o processo de inicialização **(POST – Power-On Self Test)**, a placa-mãe utiliza **códigos sonoros (bipes)** e **sinais visuais/físicos (LEDs, ventoinhas, reinícios em loop, cheiro de queimado)** para indicar falhas de hardware antes mesmo da exibição de mensagens na tela. 

---

### 🔹 Regras implementadas
O sistema especialista contempla dois tipos de diagnósticos:

- **Problemas sem bipes** → análise baseada em LEDs, ventoinhas, tela, energia e outros sinais físicos.
- **Problemas com bipes** → análise direta de acordo com tabelas de fabricantes (AMI/Gigabyte).
---

### 🔹 Justificativa de cada regra
#### 🔊 Regras com bipes
- **1 bipe → Erro de memória RAM**  
  A BIOS emite um bipe curto para indicar falha nos módulos de memória principal.

- **2 bipes → Erro de paridade da memória**  
  Indica que a memória não passou no teste de verificação de paridade.

- **3 bipes → Erro no endereço de 64 KB da memória básica**  
  Aponta problema em um endereço inicial da memória, fundamental para a inicialização.

- **4 bipes → Mau funcionamento do relógio em tempo real (RTC)**  
  O RTC é responsável pelo acompanhamento do tempo e armazenamento de informações de sistema; sua falha impede a configuração correta.

- **5 bipes → Erro na CPU**  
  Indica falha no processador, impossibilitando a execução de instruções básicas.

#### 🖥️ Regras sem bipes

- **Reinicia em loop sem vídeo**
Indica falha persistente de inicialização → BIOS corrompida, superaquecimento ou falha de energia.

- **Nada liga e cheiro de queimado presente**
Indica falha grave → Curto na placa-mãe (capacitores ou chipset queimados).

- **Liga, tela preta, ventoinhas não giram, mas LEDs acendem**
Indica que há energia chegando, mas falha na inicialização → Possível problema na placa de vídeo ou cabo de vídeo.

- **Nada liga: sem LEDs, sem ventoinhas, sem imagem**
Indica ausência total de energia → Fonte de alimentação queimada/desconectada ou placa-mãe queimada.

- **Liga, LEDs piscam, ventoinhas não giram e tela preta**
Indica falha de inicialização de vídeo/BIOS → Problema na placa de vídeo ou na BIOS.

Esse mapeamento foi implementado diretamente como regras (`@Rule`) no **motor de inferência**, tornando o sistema capaz de simular um diagnóstico automático.


In [121]:
# cod de erros para placa mães Gigabyte e AMI
class DiagnosticoPlacaMae(KnowledgeEngine):
    """interpreta a quantidade de bipes e fornece o diagnóstico correspondente."""
    
    #Possiveis problemas sem bipes
    @Rule(PlacaMae(liga=True, tela_preta=True, bipes=0, ventoinhas=False, leds=True))
    def problema_placa_video(self):
        print("Possível diagnóstico: Problema na placa de vídeo ou cabo de vídeo.")
        
    @Rule(PlacaMae(liga=True, tela_preta=True, bipes=0, leds=True, ventoinhas=False)) 
    def leds_pisca_sem_video(self):
        print("Possível diagnóstico: Problema na placa de vídeo ou na inicialização da BIOS.")

    @Rule(PlacaMae(liga=True, tela_preta=False, bipes=0, reinicia=True))
    def reinicia_em_loop(self):
        print("Possível diagnóstico: BIOS corrompida, superaquecimento ou falha de energia.")

    @Rule(PlacaMae(liga=False, tela_preta=False, bipes=0, leds=False, ventoinhas=False))
    def curto_placa_mae_sem_cheiro_queimado(self):
        print("Possível diagnóstico: Fonte de alimentação queimada/desconectada, curto na placa-mãe")

    @Rule(PlacaMae(liga=False, tela_preta=False, bipes=0, cheiro_queimado=True))
    def curto_placa_mae(self):
        print("Possível diagnóstico: Curto na placa-mãe (capacitores ou chipset queimados).")
    
    # Problemas com bipes
    @Rule(PlacaMae(bipes=1))
    def erro_memoria(self):
        print("Diagnóstico: Memory Error. Erro de memória")

    @Rule(PlacaMae(bipes=2))
    def erro_paridade_memoria(self):
        print("Diagnóstico: Memory parity check error. Erro de checagem na paridade da memória")

    @Rule(PlacaMae(bipes=3))
    def erro_endereco_memoria(self):
        print("Diagnóstico: basic memory 64K address check error. Erro de checagem do endereço de 64k da memória básica")

    @Rule(PlacaMae(bipes=4))
    def erro_relogio(self):
        print("Diagnóstico: Real Time Clock malfunction. Mau funcionamento do relógio em tempo real")

    @Rule(PlacaMae(bipes=5))
    def erro_cpu(self):
        print("Diagnóstico: CPU error. Erro na CPU")

## 3 - Rodando o Motor de inferencia

### Exemplos de diagnósticos:

In [ ]:
engine = DiagnosticoPlacaMae()
engine.reset()

#exemplos sem a placa mãe emitindo bipes
engine.declare(PlacaMae(liga=False, tela_preta=True, bipes=0, ventoinhas=False, leds=False))
engine.declare(PlacaMae(liga=True, tela_preta=True, bipes=0, ventoinhas=False, leds=True))

#exemplos com a placa mãe emitindo bipes
engine.declare(PlacaMae(bipes=2))
engine.declare(PlacaMae(bipes=5))

engine.run()
    
  

Diagnóstico: CPU error. Erro na CPU
Diagnóstico: Memory parity check error. Erro de checagem na paridade da memória
Possível diagnóstico: Problema na placa de vídeo ou cabo de vídeo.
Possível diagnóstico: Problema na placa de vídeo ou na inicialização da BIOS.


### Diagnóstico interativo: 

In [126]:
def perguntar_booleano(pergunta):
    while True:
        resp = input(pergunta + " (s/n): ").strip().lower()
        if resp in ["s", "n"]:
            return resp == "s"
        print("Digite apenas 's' ou 'n'.")

dados = {}

if perguntar_booleano("A placa-mãe está emitindo sons de bipes?"):
    while True:
        try:
            n_bipes = int(input("Digite o número de bipes (1 a 5): "))
            if 1 <= n_bipes <= 5:
                dados["bipes"] = n_bipes
                break
            else:
                print("Número inválido, tente novamente.")
        except ValueError:
            print("Digite um número válido.")
else:
    dados["bipes"] = 0
    dados["liga"] = perguntar_booleano("O computador liga?")
    dados["tela_preta"] = perguntar_booleano("A tela permanece preta?")
    dados["ventoinhas"] = perguntar_booleano("As ventoinhas giram?")
    dados["leds"] = perguntar_booleano("Os LEDs acendem?")
    dados["reinicia"] = perguntar_booleano("O PC reinicia em loop?")
    dados["cheiro_queimado"] = perguntar_booleano("Você percebe cheiro de queimado?")

    # Rodar motor
engine = DiagnosticoPlacaMae()
engine.reset()
engine.declare(PlacaMae(**dados))
engine.run()



Número inválido, tente novamente.
Digite um número válido.
Diagnóstico: CPU error. Erro na CPU
